In [ ]:
import open3d as o3d
import os
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
import math
import json
import random
import uuid
import ifcopenshell
from ifcopenshell import template
import ifcopenshell.geom

from src.visualisation import *
from src.ifc import *
from src.elements import create_pipe, create_elbow, setup_ifc_file

#create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)


In [ ]:
random.seed(10)

### CLOI Dataset Creation

The following section converts CLOI scans into a pcd dataset.

In [ ]:
# combine clouds
data_path = "/mnt/f/datasets/export/export/"
max_points = 4096

In [ ]:
classes = os.listdir(data_path)
print(classes)

In [ ]:
all_classes = []
element_count = 0
error_count = 0
for i, cl in enumerate(classes):
    all_elements = []
    class_path = data_path + cl
    elements = os.listdir(class_path)
    for j, el in tqdm(enumerate(elements)):
        try:
            element = np.loadtxt(class_path + '/' + el)

            # downsample
            if (len(element) > 0 and element.ndim == 2 and element.shape[1] == 4):
                if len(element)  > max_points:
                    #idx = np.random.randint(element.shape[0], size=max_points)
                    #element = element[idx :]
                    element = np.random.permutation(element)[:max_points]

                element = np.delete(element, 3, axis=1) # remove point index
                element = np.insert(element, 3, values=[element_count], axis=1) # add element index
                #print(element.shape)
                element_count += 1
                all_elements.append(element)
        except Exception as E:
            error_count += 1
            
            
    all_elements = np.vstack(all_elements)
    all_elements = np.insert(all_elements, 4, values=[i], axis=1) # add class index
    all_classes.append(all_elements)
    
all_classes = np.concatenate(all_classes)
print(all_classes.shape)
print ("errors: ", error_count)
        
        #print(points[0])

In [ ]:
pcd = o3d.t.geometry.PointCloud()
points = all_classes[:,0:3]
el_index  = [[i] for i in all_classes[:,3]]
cl_index  = [[i] for i in all_classes[:,4]]


pcd.point["positions"] = o3d.core.Tensor(points)
pcd.point["elements"] = o3d.core.Tensor(el_index)
pcd.point["classes"] = o3d.core.Tensor(cl_index)

o3d.t.io.write_point_cloud("water2.pcd", pcd)


## Pipe parameter detection

### Generation of synethetic IFC element dataset

#### dataset creation process
1. Generate params for element model
2. Generate ifc models
4. Convert to obj models using ifcConvert (ifc2obj.py script) 
3. Convert to partially occluded EXR images using render_depth.py script *./blender -b -P render_depth.py ../../documents/ifc/ifcopenshell-notebooks/output/obj/ output/*
4. Covnert to point clouds using process_exr.py script *python process_exr.py output/exr/ output/intrinsics.txt output/*
5. Combine multiple views of object to create training and testing datasets


##### Step 1 & 2. IFC model generation


In [ ]:
density = 1024
sample_size = 16
config_path = "config/pipeline.json"
pcd_path = "/home/haritha/downloads/blender-2.79-linux-glibc219-x86_64/output/pcd/"
blueprint = 'data/sample.ifc'
num_scans = 16

**elbow - modelled as an IfcRevolvedAreaSolid**

*params:*

position - 3D coordinate

direction - 3D vector, axis of extrusion (normal to axis of revolution) (z>=0)

axis_position - 2D coordinate, relative to position

angle - angle of revolution (0 -> pi)

radius




**pipe - modelled as an IfcExtrudedAreaSolid**

*params:*
    
position - 3D coordinate

extrusion_direction - 3D vector (z>=0)

length - 3D coordinate

radius



In [ ]:
synthetic_dataset(config_path, sample_size, "elbow", 'output', blueprint, 0)

*Use external scripts to convert above IFC dataset into ocluded PCD dataset. (step 3, 4 & 5)*

##### Step 6. Test / train dataset creation

1. merge multiple views
2. sample to standard density
3. generate training and testing dataset



In [ ]:
create_merged_dataset(pcd_path, 'output', 'tee', num_scans, density, 3, 0.1)